In [27]:
import os
import re
import sys
import html
import json
import math
import time
import threading
import subprocess
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta, timezone

#默认参数
default_config = {
    "retry_count": 5,
    "url": "http://127.0.0.1:8000",
    "title": "Podflow",
    "filename": "YouTube",
    "link": "https://m.youtube.com",
    "description": "在YouTube 上畅享您喜爱的视频和音乐上传原创内容并与亲朋好友和全世界观众分享您的视频。",
    "icon": "https://raw.githubusercontent.com/gruel-zxz/podflow/main/Podflow.png",
    "category": "TV &amp; Film",
    "channelid_youtube": {
        "youtube": {
            "update_size": 15,
            "id": "UCBR8-60-B28hp2BmDPdntcQ",
            "title": "YouTube",
            "quality": "480",
            "last_size": 50,
            "media": "m4a",
            "DisplayRSSaddress": False,
            "InmainRSS": True
        }
    }
}
# 如果InmainRSS为False或频道有更新则无视DisplayRSSaddress的状态, 都会变为True。

In [28]:
# 文件保存模块
def file_save(content, file_name, folder=None):
    # 如果指定了文件夹则将文件保存到指定的文件夹中
    if folder:
        file_path = os.path.join(os.path.join(os.getcwd(), folder), file_name)
    else:
        # 如果没有指定文件夹则将文件保存在当前工作目录中
        file_path = os.path.join(os.getcwd(), file_name)
    # 保存文件
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(content)

In [29]:
#日志模块
def write_log(log, suffix = None, display = True):
    # 获取当前的具体时间
    current_time = datetime.now()
    # 格式化输出, 只保留年月日时分秒
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    # 打开文件, 并读取原有内容
    try:
        with open("log.txt", "r") as file:
            contents = file.read()
    except FileNotFoundError:
        contents = ""
    # 将新的日志内容添加在原有内容之前
    log_in = re.sub(r"\033\[[0-9;]+m", "", log)
    new_contents = f"{formatted_time} {log_in}\n{contents}"
    # 将新的日志内容写入文件
    file_save(new_contents, "log.txt")
    if display:
        formatted_time_mini = current_time.strftime("%H:%M:%S")
        if suffix:
            print(f"{formatted_time_mini}|{log}|{suffix}")
        else:
            print(f"{formatted_time_mini}|{log}")

In [30]:
# 查看requests模块是否安装
ffmpeg_worry = '''\033[0mFFmpeg安装方法:
Ubuntu:
\033[32msudo apt update
sudo apt install ffmpeg\033[0m
CentOS:
\033[32msudo yum update
sudo yum install ffmpeg\033[0m
Debian:
\033[32msudo apt-get update
sudo apt-get install ffmpeg\033[0m
Arch Linux、Fedora:
\033[32msudo pacman -S ffmpeg
sudo dnf install ffmpeg\033[0m
检查FFmpeg版本:
\033[32mffmpeg -version\033[0m'''
try:
    # 执行 ffmpeg 命令获取版本信息
    result = subprocess.run(['ffmpeg', '-version'], capture_output = True, text = True)
    output = result.stdout.lower()
    # 检查输出中是否包含 ffmpeg 版本信息
    if 'ffmpeg version' not in output:
        write_log("FFmpeg 未安装, 请安装后重试")
        print(ffmpeg_worry)
        sys.exit(0)
except FileNotFoundError:
        write_log("FFmpeg 未安装, 请安装后重试")
        print(ffmpeg_worry)
        sys.exit(0)

# 查看requests模块是否安装并安装
try:
    import requests
    # 如果导入成功你可以在这里使用requests库
except ImportError:
    try:
        subprocess.run(['pip', 'install', 'chardet' , '-U'], capture_output = True, text = True)
        subprocess.run(['pip', 'install', 'requests' , '-U'], capture_output = True, text = True)
        write_log("\033[31mrequests安装成功, 请重新运行\033[0m")
        sys.exit(0)
    except FileNotFoundError:
        write_log("\033[31mrequests安装失败请重试\033[0m")
        sys.exit(0)

In [31]:
# HTTP GET请求重试模块
def get_with_retry(url, name, max_retries = 10, retry_delay = 6):
    for num in range(max_retries):
        try:
            response = requests.get(f"{url}")
            response.raise_for_status()
        except Exception:
            print(f"{datetime.now().strftime('%H:%M:%S')}|{name}|\033[31m连接异常重试中...\033[97m{num + 1}\033[0m")
        else:
            return response
        time.sleep(retry_delay)
    print(f"{datetime.now().strftime('%H:%M:%S')}|{name}|\033[31m达到最大重试次数\033[97m{max_retries}\033[0m")
    return None

In [42]:
# 安装库模块
def library_install(library ,library_install_dic = None):
    if version := re.search(
        r"(?<=Version\: ).+",
        subprocess.run(
            ["pip", "show", library], capture_output = True, text = True
        ).stdout
    ):
        write_log(f"{library}已安装")
        if library_install_dic:
            version_update = library_install_dic[library]
        else:
            # 获取最新版本编号
            version_update = get_with_retry(f"https://pypi.org/project/{library}/", f"{library}", 2, 2)
            if version_update:
                version_update = re.search(
                    r"(?<=<h1 class=\"package-header__name\">).+?(?=</h1>)",
                    version_update.text,
                    flags=re.DOTALL
                )
        # 如果库已安装, 判断是否为最新
        if version_update is None or version.group() not in version_update.group():
            # 如果库已安装, 则尝试更新
            try:
                subprocess.run(['pip', 'install', '--upgrade', library], capture_output = True, text = True)
                write_log(f"{library}更新成功|版本：\033[32m{version.group()}\033[0m")
            except FileNotFoundError:
                write_log(f"{library}更新失败")
        else:
            write_log(f"{library}无需更新|版本：\033[32m{version.group()}\033[0m")
    else:
        write_log(f"{library}未安装")
        # 如果库未安装, 则尝试安装
        try:
            subprocess.run(['pip', 'install', library , '-U'], capture_output = True, text = True)
            write_log(f"{library}安装成功")
        except FileNotFoundError:
            write_log(f"{library}安装失败")

In [43]:
# 安装/更新并加载三方库
library_install_list = ["yt-dlp", "RangeHTTPServer", "chardet", "requests", "astral"]
library_install_dic = {}
def library_install_get(library):
    # 获取最新版本编号
    version_update = get_with_retry(f"https://pypi.org/project/{library}/", f"{library}", 2, 2)
    if version_update:
        version_update = re.search(
            r"(?<=<h1 class=\"package-header__name\">).+?(?=</h1>)",
            version_update.text,
            flags=re.DOTALL
        )
        if version_update:
            library_install_dic[library] = version_update
# 创建线程列表
library_install_get_threads = []
for library in library_install_list:
    thread = threading.Thread(target=library_install_get, args=(library,))
    library_install_get_threads.append(thread)
    thread.start()
# 等待所有线程完成
for thread in library_install_get_threads:
    thread.join()
print(library_install_dic)
for library in library_install_list:
    library_install(library ,library_install_dic)



{'requests': <re.Match object; span=(8360, 8391), match='\n        requests 2.31.0\n      '>, 'RangeHTTPServer': <re.Match object; span=(8470, 8507), match='\n        rangehttpserver 1.3.3\n      '>, 'chardet': <re.Match object; span=(8394, 8423), match='\n        chardet 5.2.0\n      '>, 'yt-dlp': <re.Match object; span=(8412, 8445), match='\n        yt-dlp 2023.10.13\n      '>, 'astral': <re.Match object; span=(8410, 8436), match='\n        astral 3.2\n      '>}
03:51:33|yt-dlp已安装
03:51:33|yt-dlp无需更新|版本：2023.10.13
03:51:34|RangeHTTPServer已安装
03:51:34|RangeHTTPServer无需更新|版本：1.3.3
03:51:36|chardet已安装
03:51:36|chardet无需更新|版本：5.2.0
03:51:37|requests已安装
03:51:37|requests无需更新|版本：2.31.0
03:51:38|astral已安装
03:51:38|astral无需更新|版本：3.2
